In [85]:
import pandas as pd
import math



Read original file

In [84]:
# Replace 'your_file.csv' with the path to your CSV file
file_path = './datathon_2024_dataset_corrected.csv'

# Read the CSV file into a pandas DataFrame
original_df = pd.read_csv(file_path)

print(original_df.head())

   Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
0           0   20000329       NYN       CHN        False           3   
1           1   20000330       CHN       NYN        False           1   
2           2   20000403       ATL       COL         True           2   
3           3   20000403       CIN       MIL         True           3   
4           4   20000403       FLO       SFN        False           6   

   away_score  venue        venue_name        city  ... away_hbp  home_pa  \
0           5  TOK01        Tokyo Dome       Tokyo  ...        1       38   
1           5  TOK01        Tokyo Dome       Tokyo  ...        1       44   
2           0  ATL02      Turner Field     Atlanta  ...        0       32   
3           3  CIN08     Cinergy Field  Cincinnati  ...        0       20   
4           4  MIA01  Sun Life Stadium       Miami  ...        0       38   

   home_1b  home_2b  home_3b  home_hr  home_fo  home_so  home_bb  home_hbp  
0        5        1  

Read location file

In [86]:
location_file_path = './cities_coordinates.xlsx'
location_df = pd.read_excel(location_file_path)
print(location_df.head())

         city  latitude  longitude
0     Atlanta   33.7490   -84.3880
1  Cincinnati   39.1031   -84.5120
2       Miami   25.7617   -80.1918
3    Montreal   45.5017   -73.5673
4    New York   40.7128   -74.0060


Function to calculate the distance with latitudes and longitudes.

In [87]:
# Calculate the distance between two points.

def haversine(lat1, lon1, lat2, lon2):
    # 将经纬度转换为弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # 地球半径（单位：公里）
    radius = 6371.0

    # 计算距离
    distance = radius * c

    return distance

# 示例：两个地点的经纬度
lat1, lon1 = 37.7749, -122.4194  # 地点1（例如，旧金山）
lat2, lon2 = 34.0522, -118.2437  # 地点2（例如，洛杉矶）

# 计算距离
result = haversine(lat1, lon1, lat2, lon2)
print(f"distance is：{result:.2f} ")


distance is：559.12 


Delete outlier cities.

In [5]:
#print out all city names

# Delelete all rows with city "Tokyo", "London", "Sydney",  "San Juan", "Monterrey"
original_df = original_df[original_df['city'] != 'Tokyo']
original_df = original_df[original_df['city'] != 'London']
original_df = original_df[original_df['city'] != 'Sydney']
original_df = original_df[original_df['city'] != 'San Juan']
original_df = original_df[original_df['city'] != 'Monterrey']

# print(original_df['city'].unique())

# Count the number of unique city
# print(original_df['city'].nunique())



Add location to original file.

In [88]:
import pandas as pd

# Assuming both venue_name columns are of the same data type


# Merge the two DataFrames based on 'venue_name'
merged_df = pd.merge(original_df, location_df, on='city', how='inner')

# 'inner' will keep only the rows where venue_name exists in both tables

# Display the merged DataFrame

# Sort merged_df by game_date
merged_df = merged_df.sort_values(by='game_date')
print(merged_df.head(20))



       Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
0               2   20000403       ATL       COL         True           2   
11692           8   20000403       ANA       NYA        False           2   
13586           9   20000403       BAL       CLE         True           1   
15476          10   20000403       MIN       TBA        False           0   
6017            6   20000403       NYN       SDN         True           2   
5656            5   20000403       MON       LAN        False           4   
17369          11   20000403       OAK       DET        False           4   
19257          12   20000403       TEX       CHA         True          10   
21152          13   20000403       TOR       KCA         True           5   
3781            4   20000403       FLO       SFN        False           6   
1891            3   20000403       CIN       MIL         True           3   
9802            7   20000403       SLN       CHN         True           7   

Calculate distance from last game.

In [89]:
# Initialize the distance columns
merged_df['home_team_distance'] = 0
merged_df['away_team_distance'] = 0

# Dictionary to keep track of the last location of each team
last_locations = {}

# Iterate through each row in the DataFrame
for index, row in merged_df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    current_location = (row['latitude'], row['longitude'])
    
    # For home team
    if home_team in last_locations:
        # Calculate the distance from the last location
        last_location = last_locations[home_team]
        distance = haversine(current_location[0], current_location[1], last_location[0], last_location[1])
        merged_df.at[index, 'home_team_distance'] = distance
    last_locations[home_team] = current_location

    # For away team
    if away_team in last_locations:
        # Calculate the distance from the last location
        last_location = last_locations[away_team]
        distance = haversine(current_location[0], current_location[1], last_location[0], last_location[1])
        merged_df.at[index, 'away_team_distance'] = distance
    last_locations[away_team] = current_location

# Display the DataFrame
print(merged_df.head(50))



       Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
0               2   20000403       ATL       COL         True           2   
11692           8   20000403       ANA       NYA        False           2   
13586           9   20000403       BAL       CLE         True           1   
15476          10   20000403       MIN       TBA        False           0   
6017            6   20000403       NYN       SDN         True           2   
5656            5   20000403       MON       LAN        False           4   
17369          11   20000403       OAK       DET        False           4   
19257          12   20000403       TEX       CHA         True          10   
21152          13   20000403       TOR       KCA         True           5   
3781            4   20000403       FLO       SFN        False           6   
1891            3   20000403       CIN       MIL         True           3   
9802            7   20000403       SLN       CHN         True           7   

In [90]:
# output a new csv file
merged_df.to_csv('new_data.csv', index=False)

In [91]:
merged_df['game_date'] = pd.to_datetime(merged_df['game_date'], format='%Y%m%d')
print(merged_df.head(50))


       Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
0               2 2000-04-03       ATL       COL         True           2   
11692           8 2000-04-03       ANA       NYA        False           2   
13586           9 2000-04-03       BAL       CLE         True           1   
15476          10 2000-04-03       MIN       TBA        False           0   
6017            6 2000-04-03       NYN       SDN         True           2   
5656            5 2000-04-03       MON       LAN        False           4   
17369          11 2000-04-03       OAK       DET        False           4   
19257          12 2000-04-03       TEX       CHA         True          10   
21152          13 2000-04-03       TOR       KCA         True           5   
3781            4 2000-04-03       FLO       SFN        False           6   
1891            3 2000-04-03       CIN       MIL         True           3   
9802            7 2000-04-03       SLN       CHN         True           7   

Add rest time and consecutive days of playing

In [92]:
# Function to calculate last rest day and consecutive playing days
def calculate_playing_info(team_games):
    sorted_dates = sorted(team_games)
    rest_days = {}
    consecutive_days = {}
    last_date = None
    consecutive_count = 0

    for date in sorted_dates:
        if last_date and date - last_date == timedelta(days=1):
            consecutive_count += 1
        else:
            consecutive_count = 1

        if last_date and date - last_date > timedelta(days=1):
            rest_day = date - timedelta(days=1)
        else:
            rest_day = None

        rest_days[date] = rest_day
        consecutive_days[date] = consecutive_count
        last_date = date

    return rest_days, consecutive_days

# Create a dictionary to store all game dates for each team
team_games = {}
for index, row in merged_df.iterrows():
    for team in [row['home_team'], row['away_team']]:
        if team not in team_games:
            team_games[team] = []
        team_games[team].append(row['game_date'])

# Calculate rest days and consecutive playing days for each team
team_info = {team: calculate_playing_info(dates) for team, dates in team_games.items()}

# Update the DataFrame with the calculated data
for index, row in merged_df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    game_date = row['game_date']

    # Update info for home team
    merged_df.at[index, 'home_last_rest_day'] = team_info[home_team][0].get(game_date)
    merged_df.at[index, 'home_consecutive_playing_days'] = team_info[home_team][1].get(game_date)

    # Update info for away team
    merged_df.at[index, 'away_last_rest_day'] = team_info[away_team][0].get(game_date)
    merged_df.at[index, 'away_consecutive_playing_days'] = team_info[away_team][1].get(game_date)

In [93]:
print(merged_df[(merged_df['home_team'] == 'HOU') | (merged_df['away_team'] == 'HOU')].head(20))


       Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
24858          19 2000-04-04       PIT       HOU        False           2   
24859          32 2000-04-05       PIT       HOU        False           2   
24860          46 2000-04-06       PIT       HOU         True          10   
28640          57 2000-04-07       HOU       PHI        False           1   
28641          72 2000-04-08       HOU       PHI         True           8   
28642          87 2000-04-09       HOU       PHI         True           2   
28643          99 2000-04-10       HOU       SLN        False           7   
28644         107 2000-04-11       HOU       SLN        False           6   
28645         119 2000-04-12       HOU       SLN        False           7   
43784         150 2000-04-14       SDN       HOU        False           4   
43785         163 2000-04-15       SDN       HOU        False           5   
43786         179 2000-04-16       SDN       HOU         True          13   

In [94]:
from timezonefinder import TimezoneFinder
import pytz
from datetime import datetime

# Function to get the timezone from latitude and longitude
def get_timezone(latitude, longitude):
    tf = TimezoneFinder()
    timezone_str = tf.timezone_at(lat=latitude, lng=longitude)
    return timezone_str

# Function to calculate the time difference between two timezones
def calculate_time_diff(tz1, tz2):
    if not tz1 or not tz2:
        return 0
    current_time = datetime.utcnow()
    tz1_time = pytz.timezone(tz1).localize(current_time)
    tz2_time = pytz.timezone(tz2).localize(current_time)
    return int((tz1_time.utcoffset() - tz2_time.utcoffset()).total_seconds() / 3600)

# Precompute timezones for each unique location
unique_locations = merged_df[['latitude', 'longitude']].drop_duplicates()
unique_locations['timezone'] = unique_locations.apply(lambda row: get_timezone(row['latitude'], row['longitude']), axis=1)
location_timezones = dict(zip(unique_locations[['latitude', 'longitude']].apply(tuple, axis=1), unique_locations['timezone']))

# Initialize the jet lag columns
merged_df['home_team_jet_lag'] = 0
merged_df['away_team_jet_lag'] = 0

# Dictionary to keep track of the last location of each team
last_locations_timezones = {}

# Iterate through each row in the DataFrame
for index, row in merged_df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    current_location = (row['latitude'], row['longitude'])
    current_timezone = location_timezones[current_location]

    # Calculate jet lag for home team
    if home_team in last_locations_timezones:
        last_timezone = last_locations_timezones[home_team]
        merged_df.at[index, 'home_team_jet_lag'] = calculate_time_diff(current_timezone, last_timezone)
    last_locations_timezones[home_team] = current_timezone

    # Calculate jet lag for away team
    if away_team in last_locations_timezones:
        last_timezone = last_locations_timezones[away_team]
        merged_df.at[index, 'away_team_jet_lag'] = calculate_time_diff(current_timezone, last_timezone)
    last_locations_timezones[away_team] = current_timezone

# Display the DataFrame
print(merged_df.head(20))

       Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
0               2 2000-04-03       ATL       COL         True           2   
11692           8 2000-04-03       ANA       NYA        False           2   
13586           9 2000-04-03       BAL       CLE         True           1   
15476          10 2000-04-03       MIN       TBA        False           0   
6017            6 2000-04-03       NYN       SDN         True           2   
5656            5 2000-04-03       MON       LAN        False           4   
17369          11 2000-04-03       OAK       DET        False           4   
19257          12 2000-04-03       TEX       CHA         True          10   
21152          13 2000-04-03       TOR       KCA         True           5   
3781            4 2000-04-03       FLO       SFN        False           6   
1891            3 2000-04-03       CIN       MIL         True           3   
9802            7 2000-04-03       SLN       CHN         True           7   

In [78]:
# print ouly HOU for me home_team or away_team
print(merged_df[(merged_df['home_team'] == 'HOU') | (merged_df['away_team'] == 'HOU')].head(20))

       Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
24858          19 2000-04-04       PIT       HOU        False           2   
24859          32 2000-04-05       PIT       HOU        False           2   
24860          46 2000-04-06       PIT       HOU         True          10   
28640          57 2000-04-07       HOU       PHI        False           1   
28641          72 2000-04-08       HOU       PHI         True           8   
28642          87 2000-04-09       HOU       PHI         True           2   
28643          99 2000-04-10       HOU       SLN        False           7   
28644         107 2000-04-11       HOU       SLN        False           6   
28645         119 2000-04-12       HOU       SLN        False           7   
43784         150 2000-04-14       SDN       HOU        False           4   
43785         163 2000-04-15       SDN       HOU        False           5   
43786         179 2000-04-16       SDN       HOU         True          13   

In [64]:
# plot a graph showing home_team_distance over time of the team 'HOU'(home_team or away_team)
import matplotlib.pyplot as plt

# Filter the DataFrame for the team 'HOU'
HOU_df = merged_df[(merged_df['home_team'] == 'HOU') | (merged_df['away_team'] == 'HOU')]
HOU_df = HOU_df.sort_values(by='game_date')

# Use plotly to plot the graph
import plotly.express as px

fig = px.line(HOU_df, x='game_date', y='home_team_distance', title='Distance Traveled for HOU')
fig.show()
